In [ ]:
from __future__ import print_function
import math

from IPython.display import display
from matplotlib import pyplot as plt
from sklearn import metrics

import time
import numpy as np
import pandas as pd
import tensorflow as tf


tf.logging.set_verbosity(tf.logging.ERROR)

train_data = pd.read_csv("./dataSet/train.csv")
test_data = pd.read_csv("./dataSet/test.csv")

train_copy = test_data.copy()
test_copy = test_data.copy()

In [ ]:
display(train_data.info())
print(int(time.time()))

In [ ]:
def construct_features(data):
    return (set([tf.feature_column.numeric_column(feature) for feature in data]))        
    
def train(data, mod_directory=None, learning_rate=0.01, batch_size=5, steps=100, periods=10):
    # Random data set
    train_set = data.sample(frac=0.8, replace=False, random_state=100)
    cv_set = data.loc[set(data.index) - set(train_set.index)]
    feature_columns = construct_features(data.drop('label', axis=1))
    
    # Set input function
    train_input = tf.estimator.inputs.pandas_input_fn(
        x = train_set.drop('label', axis=1),
        y = train_set.label,
        queue_capacity=batch_size,
        num_epochs=None,
        shuffle=True,
        batch_size=batch_size
    )
    
    train_eval_input = tf.estimator.inputs.pandas_input_fn(
        x = train_set.drop('label', axis=1),
        y = train_set.label,
        num_epochs=1,
        shuffle=True,
    )
    
    cv_input = tf.estimator.inputs.pandas_input_fn(
        x = cv_set.drop('label', axis=1),
        y = cv_set.label,
        num_epochs=1,
        shuffle=False,
    )
            
    # Build DNN classifier
    estimator = tf.estimator.DNNClassifier(
        hidden_units=[1024, 512, 64],
        feature_columns=feature_columns,
        optimizer=tf.train.AdamOptimizer(learning_rate),
        n_classes=10,
        dropout=0.1,
        model_dir=mod_directory + str(int(time.time())) if mod_directory else None,
#         warm_start_from='./test1551856412',
    )
    
    training_rmse = []
    validation_rmse = []
    
    for i in range(periods):
        print('%d period:' % (i + 1), end='')
        estimator.train(input_fn=train_input, steps=steps)
            
        # Evaluate training 
        eval_train = estimator.evaluate(input_fn=cv_input) 
        print(eval_train)
            
    print("Model training finished.")
    return estimator



In [ ]:
estimator1 = train(
    train_data,
    learning_rate=1e-4,
    batch_size=50,
    steps=100,
    periods=30,
    mod_directory='test',
)